<a href="https://colab.research.google.com/github/semkud/NLP/blob/main/HW1/NLP_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импортируем все, что нам понадобится

In [133]:
from bs4 import BeautifulSoup
import re
import nltk
import pymorphy2
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from nltk.tokenize import word_tokenize
import collections
import random

Эта функция принимает html-страницу с отзывами, токенизирует их по словам, и возвращает частотный словарь слов и 10 отзывов, на которых будет происходить проверка.

In [134]:
def tokening(filename):
  with open(filename, encoding='utf-8') as f:
        html = f.read()

  mylist = []   
  soup = BeautifulSoup(html,'html.parser')
  for post in soup.find_all('div', {'class': 'text use-default-ui'}):
    mylist.append(post.get_text()) #Пока у нас список строк, где каждый элемент один отзыв

  print('всего отзывов - ', len(mylist))
  
  for_teaching = []
  for_testing = []  
  random.shuffle(mylist) #Перемешаем для чистоты эксперимента
  for_teaching = mylist[:-10] # Возьмем все без 10 для обучения
  for_testing = mylist[-10:] # И 10 последних для проверки

  Tokens = []
  for review in for_teaching:
    a = word_tokenize(review)
    words = [w.lower() for w in a if w.isalpha()] #приводим к нижнему регистру и выбрасываем все неслова
    for word in words:
      Tokens.append((morph.parse(word)[0].normal_form)) #записываем начальную форму в список
  counter = collections.Counter(Tokens)
  return counter, for_testing

Функция, которая составляет списки слов, которые встречаются только в положительных или только отрицательных отзывах

In [135]:
def distillate(counter1, counter2, mylist):
  for word in counter1:
    if not word in counter2:
      mylist.append(word)

Функция, которая проверяет отзыв по словам, чтобы определить, положительный он или отрицательный.

In [141]:
def testing(list, q):
  Accuracy = []
  for review in list:
    
    init_form = []
    karma = 0
    a = word_tokenize(review)
    words = [w.lower() for w in a if w.isalpha()]
    for word in words:
      init_form.append((morph.parse(word)[0].normal_form)) #стокенизировали отзыв

    for form in init_form:
      if form in veryGood:
        karma = karma + 1
      else:
        if form in veryBad:
          karma = karma -1 #посчитали каких слов больше встретилось
    #print(review, karma) #можно распечатать пару отзыв - оценка его
    if q is '+': # Если второй аргумент функции - +, то есть сейчас мы тестируем положительные отзывы, то мы "справились" ,если карма больше 0
      if karma > 0:
        Accuracy.append(karma)
    else:
      if karma < 0:
        Accuracy.append(karma)
  print ('Точность предсказания - ', len(Accuracy)/len(list))

Создали некоторые списки

In [137]:
Goodrev = [] #Список, в который мы положим отзывы, на которых будем проверять нашу программу
Badrev = []
veryGood = [] #Слова, которые встречаются только в положительных отзывах
veryBad = []

Поехали

In [138]:
BadR, Badrev = tokening('ОтзывыПлохие.html')
GoodR, Goodrev = tokening('ОтзывыХорошие.html')

всего отзывов -  330
всего отзывов -  330


In [139]:
distillate(BadR, GoodR, veryBad)
distillate(GoodR, BadR, veryGood)

In [142]:
testing(Badrev, '-')
testing(Goodrev, '+')

Точность предсказания -  1.0
Точность предсказания -  0.5
